In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook
import bs4
import re
import os
from requests.exceptions import ConnectionError
from urllib3.exceptions import ProtocolError
from http.client import RemoteDisconnected
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

from sofa_score_scrap import *

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}

In [158]:
de_para_siglas = pd.read_excel("de_para_siglas_epl.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

In [29]:
# s.get('https://www.sofascore.cddom/pt/atletico-mineiro-internacional/qOsCO', headers={'x-test2': 'true'})
resp = requests.get("https://www.sofascore.com/event/7724681/statistics/players/json?_=154328071", headers=headers)

In [30]:
players_df = pd.DataFrame(resp.json()["players"])

In [35]:
get_per_player_data(players_df, de_para_siglas)

,name,slug,shortName,id,disabled,national,team_name,team_id,goals,goalAssist,...,totalLongBalls,possessionLost,fouls,wasFouled,offsides,saves,punches,runsOut,goodHighClaim,team
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,Juan Cazares,atletico-mineiro,J. Cazares,158411,False,False,Atlético Mineiro,1977,1,1,...,2 (2),17,2,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Andres D'Alessandro,internacional,A. D'Alessandro,17639,NaN,False,Internacional,1966,1,,...,5 (2),23,1,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Víctor Cuesta,internacional,V. Cuesta,587148,NaN,False,Internacional,1966,,,...,11 (7),13,2,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,David Terans,atletico-mineiro,D. Terans,586662,False,False,Atlético Mineiro,1977,1,,...,2 (1),5,1,0,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Nico López,internacional,N. López,255881,NaN,False,Internacional,1966,,,...,7 (4),15,1,1,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Patric,atletico-mineiro,Patric,78853,False,False,Atlético Mineiro,1977,,,...,8 (1),22,1,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Luan,atletico-mineiro,Luan,243427,False,False,Atlético Mineiro,1977,,,...,1 (0),8,4,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Emerson Santos,internacional,E. Santos,844015,NaN,False,Internacional,1966,,,...,4 (2),9,0,0,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Fabiano,internacional,F. Leismann,551304,NaN,False,Internacional,1966,,,...,5 (2),20,2,0,NaN,NaN,NaN,NaN,NaN,INT


### Game Stats

In [43]:
resp = requests.get("https://www.sofascore.com/event/7724681/general/json", headers=headers)

In [38]:
pd.DataFrame(game_statistics(resp, players_df, de_para_siglas, period=0), index=[0]).set_index("game")

,team_away,team_home,data,hora,away_score,home_score,result,Ball possession_home,Ball possession_away,Total shots_home,...,form_minute_82,form_minute_83,form_minute_84,form_minute_85,form_minute_86,form_minute_87,form_minute_88,form_minute_89,form_minute_90,form_minute_90.5
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,CAM,INT,2018-11-21,19:30:00,2,1,-1,54%,46%,15,...,22,73,41,24,1,1,63,29,28,-49


In [48]:
resp.json()["statistics"]["periods"][0]['period']

'ALL'

### Pega links dos jogos históricos

#### Pega os IDs das temporadas

In [4]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/spain/laliga/8", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/bundesliga/35", headers=headers)
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/italy/serie-a/23", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/2-bundesliga/44", headers=headers)
c = bs4.BeautifulSoup(comp.content)

In [5]:
temporadas = c.find_all("script", attrs={"id":"__NEXT_DATA__"})

In [6]:
temp = str(temporadas[0]).replace('<script id="__NEXT_DATA__" type="application/json">', '')
temp = temp.replace('</script>', '')

In [7]:
temp_j = json.loads(temp)

In [8]:
season_ids = temp_j['props']['initialProps']['pageProps']['seasons']

In [9]:
season_ids

[{'name': 'Serie A 19/20', 'year': '19/20', 'id': 24644},
 {'name': 'Serie A 18/19', 'year': '18/19', 'id': 17932},
 {'name': 'Serie A 17/18', 'year': '17/18', 'id': 13768},
 {'name': 'Serie A 16/17', 'year': '16/17', 'id': 11966},
 {'name': 'Serie A 15/16', 'year': '15/16', 'id': 10596},
 {'name': 'Serie A 14/15', 'year': '14/15', 'id': 8618},
 {'name': 'Serie A 13/14', 'year': '13/14', 'id': 6797},
 {'name': 'Serie A 12/13', 'year': '12/13', 'id': 5145},
 {'name': 'Serie A 11/12', 'year': '11/12', 'id': 3639},
 {'name': 'Serie A 10/11', 'year': '10/11', 'id': 2930},
 {'name': 'Serie A 09/10', 'year': '09/10', 'id': 2324},
 {'name': 'Serie A 08/09', 'year': '08/09', 'id': 1552},
 {'name': 'Serie A 07/08', 'year': '07/08', 'id': 712},
 {'name': 'Serie A 06/07', 'year': '06/07', 'id': 98},
 {'name': 'Serie A 05/06', 'year': '05/06', 'id': 97},
 {'name': 'Serie A 04/05', 'year': '04/05', 'id': 96},
 {'name': 'Serie A 03/04', 'year': '03/04', 'id': 95},
 {'name': 'Serie A 02/03', 'year': 

In [101]:
json.dump(season_ids, open("IDs_2nd_Bundesliga_seasons.json", 'w'))

### Temporada

In [10]:
temporadas = json.load(open("IDs_SerieA_Italiana_seasons.json", 'r'))

In [11]:
# temporadas = c.find_all("a", attrs={"class":"pointer js-uniqueTournament-page-load-season"})

id_temporada = {a['year']: a['id'] for a in temporadas}
id_temporada

{'19/20': 24644,
 '18/19': 17932,
 '17/18': 13768,
 '16/17': 11966,
 '15/16': 10596,
 '14/15': 8618,
 '13/14': 6797,
 '12/13': 5145,
 '11/12': 3639,
 '10/11': 2930,
 '09/10': 2324,
 '08/09': 1552,
 '07/08': 712,
 '06/07': 98,
 '05/06': 97,
 '04/05': 96,
 '03/04': 95,
 '02/03': 80,
 '01/02': 81,
 '00/01': 82,
 '99/00': 83,
 '98/99': 84,
 '97/98': 85}

## Pega Links dos jogos por Rodadas

#### Página Inicial

In [14]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/spain/laliga/8", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/bundesliga/35", headers=headers)
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/italy/serie-a/23", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/2-bundesliga/44", headers=headers)

In [15]:
s = requests.Session()
s.get(comp.url, headers=headers)

<Response [200]>

#### Para cada temporada

In [42]:
prefix = "ITA"
de_para_siglas = pd.read_excel("de_para_siglas_italy.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

for temp in list(id_temporada.keys())[16:]:
    ret = []
    
    for rodada in tqdm_notebook(range(1, 39)):
        # print(temp[0] + ": " + str(rodada), end="\r")
        resp = s.get("https://www.sofascore.com/u-tournament/23/season/"+ str(id_temporada[temp]) + "/matches/round/" + str(rodada), headers=headers)
        resp.encoding = 'UTF-8'
        try:
            ret.append(get_info_rodada(resp, de_para_siglas))
        except IndexError:
            pass
    
    ret = pd.concat(ret)
    ret.to_csv("links_sofa_score/sofa_score_links_"+prefix+"_"+temp.replace("/", "_")+".csv", sep=";")

KeyboardInterrupt: 

### Pega informações dos jogos históricos

In [2]:
base = pd.read_csv("links_sofa_score/sofa_score_links_EPL_18_19.csv", sep=";")

In [43]:
list(reversed(sorted([x for x in os.listdir("links_sofa_score/") if 'ITA' in x])))

['sofa_score_links_ITA_19_20.csv',
 'sofa_score_links_ITA_18_19.csv',
 'sofa_score_links_ITA_17_18.csv',
 'sofa_score_links_ITA_16_17.csv',
 'sofa_score_links_ITA_15_16.csv',
 'sofa_score_links_ITA_14_15.csv',
 'sofa_score_links_ITA_13_14.csv',
 'sofa_score_links_ITA_12_13.csv',
 'sofa_score_links_ITA_11_12.csv',
 'sofa_score_links_ITA_10_11.csv',
 'sofa_score_links_ITA_09_10.csv',
 'sofa_score_links_ITA_08_09.csv',
 'sofa_score_links_ITA_07_08.csv',
 'sofa_score_links_ITA_06_07.csv',
 'sofa_score_links_ITA_05_06.csv',
 'sofa_score_links_ITA_04_05.csv',
 'sofa_score_links_ITA_03_04.csv']

In [44]:
API_HEADERS = {
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"8b4895fbff"',
    'origin': 'https://www.sofascore.com',
    # 'referer': 'https://www.sofascore.com/pt/chelsea-manchester-city/rN'
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
}

In [45]:
prefix = 'ITA'
lista = list(reversed(sorted([x for x in os.listdir("links_sofa_score/") if prefix in x])))

In [46]:
s = requests.Session()
s.get(comp.url, headers=headers)

<Response [200]>

In [71]:

 game_id += 1

In [73]:
de_para_siglas = pd.read_excel("de_para_siglas_italy.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

erros = []

for file in lista[2:4]:
    base = pd.read_csv("links_sofa_score/" + file, sep=";")
    
    base[(base.home_score != -1) & (base.away_score != -1)]
    
    # resp_game = []
    # resp_players = []
    # resp_incidents_df = {}
    # for key in ['period', 'substitution', 'injuryTime', 'goal', 'card', 'varDecision']:
    #     resp_incidents_df[key] = []
        
    # game_id = 0
    
    # name = file[-11:-4]
    name = file[-13:-4]
    # name = file[-17:-4]
    
    tentativas = 3
    while(tentativas > 0):
        try:
            while (game_id < len(base.id)):
                print(name + " Game #" + str(game_id), end="\r")
                
                API_HEADERS['referer'] = base.link.iloc[game_id]
                
                # # # # # # # # #
                # DADOS DO JOGO #
                # # # # # # # # #
                resp = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/general/json",
                             headers=API_HEADERS, timeout=10)

                # # # # # # # # # # #
                # DADOS POR JOGADOR #
                # # # # # # # # # # #
                resp2 = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/statistics/players/json",
                              headers=API_HEADERS, timeout=10)
                resp2.encoding = 'UTF-8'
                
                # # # # # # #
                # INCIDENTS #
                # # # # # # #
                resp_incidents = s.get('https://api.sofascore.com/api/v1/event/'+ str(base.id.iloc[game_id]) +'/incidents',
                                      headers=API_HEADERS, timeout=10)
                resp_incidents.encoding = 'UTF-8'
                
                
                # time.sleep(np.random.choice([2, 3, 4, 5]))
                time.sleep(np.random.choice([0.01, 0.1, 0.5]))
                
                if "Statistics not found." in str(resp2.content):
                    erros.append(base.id.iloc[game_id])
                    tentativas = 0
                    game_id += 1
                    continue
                    
                try:
                    players_df = pd.DataFrame(resp2.json()["players"])  
                except KeyError:
                    players_df = None
                    pass
        
                ### Guarda estatisticas do jogo
                game_df = pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0])
                game_df['game'] = base.game.iloc[game_id]
                resp_game.append(
                    game_df.set_index("game")
                )
                
                ### Guarda statisticas dos jogadores
                if players_df is not None and not players_df.empty:
                    per_player_data_df = get_per_player_data(players_df, de_para_siglas)
                    per_player_data_df['game'] = base.game.iloc[game_id]
                    resp_players.append(per_player_data_df.set_index("game")) 
                
                
                ### Guarda Incidents 
                incidents_df = get_incidents_database(resp_incidents)
                for key in incidents_df:
                    # Adiciona coluna "GAME" para index
                    incidents_df[key]['game'] = base.game.iloc[game_id]
                    incidents_df[key].set_index('game', inplace=True)
                    
                    resp_incidents_df[key].append(
                        incidents_df[key].copy()
                    )
                
                tentativas = 0
                game_id += 1
        except (ConnectionError, ProtocolError, RemoteDisconnected):
            print("Error")
            time.sleep(2)
            tentativas -= 1

    print("Guardando dados capturados...")
    
    try:
        os.makedirs(prefix + "_data_sofa_score/" + name + "/")
    except:
        pass
    
    ret_game = pd.concat(resp_game)
    ret_game.drop_duplicates().to_csv(prefix + "_data_sofa_score/" + name + "/game_data_"+ name +".csv", sep=";")

    ret_players = pd.concat(resp_players)
    ret_players.drop_duplicates().to_csv(prefix + "_data_sofa_score/" + name + "/players_data_"+ name +".csv", sep=";")

    for key in resp_incidents_df:
        if len(resp_incidents_df[key]) > 0:
            resp_incidents_concat = pd.concat(resp_incidents_df[key])
            try:
                resp_incidents_concat = resp_incidents_concat.drop_duplicates()
            except: 
                pass
            resp_incidents_concat.to_csv(prefix + "_data_sofa_score/" + name + "/incidents_" + key + "_data_" + name + ".csv", sep=';')
    
    print("Com sucesso!")
    print()
    
    resp_game = []
    resp_players = []
    resp_incidents_df = {}
    for key in ['period', 'substitution', 'injuryTime', 'goal', 'card', 'varDecision']:
        resp_incidents_df[key] = []
        
    game_id = 0

IndentationError: expected an indented block (<ipython-input-73-0df2fe036dea>, line 73)